In [1]:
import pandas as pd
import json
import os

In [2]:
# Normalizar Categorías
def normalize_categories(category_list):
    '''
    Normaliza la lista de categorías
    '''
    if category_list is not None:
        return [category.strip() for category in category_list]
    else:
        return None

In [3]:
# Normalizar Horas
def normalize_hours(hours_list):
    '''
    Normaliza la lista de horas
    '''
    if hours_list is not None:
        normalized_hours = []
        for day_hours in hours_list:
            if len(day_hours) == 2:
                day, hours = day_hours
                normalized_day_hours = f"{day}: {hours}"
                normalized_hours.append(normalized_day_hours)
        return normalized_hours
    else:
        return None

In [20]:
# Normalizar Json
def normalize(j):
    '''
    Normaliza el json
    '''
    fields = ['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'state', 'url']
    df = pd.DataFrame(j, columns=fields)
    df['category'] = df['category'].apply(normalize_categories)
    df_meta = df.dropna(subset=['category'])
    df_meta = df_meta.explode('category')
    # df_meta = df_meta.drop(columns=['relative_results'])
    # df_meta = df_meta.drop(columns=['MISC'])
    df_meta['hours'] = df_meta['hours'].apply(normalize_hours)
    df_meta['hours'] = df_meta['hours'].apply(lambda x: ', '.join(x) if x is not None else None)
    return df_meta

In [38]:

# Obtener los datos del evento
# data = cloud_event.data
bucket_name = '../../assets/metadata-sites'#data["bucket"]
file_name = '0.json'#data["name"]
# Leer el json de GCS
# blob = read_json_from_gcs(bucket_name, file_name)    
category_selected = 'Tourist attraction'
# leer el archivo json linea por linea y ejecutar la función normalize
df = pd.DataFrame() 
# for line in blob.download_as_string().decode("utf-8").splitlines():
with open(bucket_name+'/'+file_name, 'r') as f:
    for line in f.readlines():   
        # Lee el archivo JSON
        j = json.loads(line)
        normalized = normalize([j])
        # Seleccionar solo las atracciones turísticas
        normalized = normalized[normalized['category'].str.contains(category_selected, case=False, na=False)]
        # Concatenar el json normalizado al dataframe
        if not normalized.empty:
            df = pd.concat([df, normalized])  
df
# # Guardar el json normalizado en BigQuery
# project_id = os.environ.get('GCP_PROJECT')
# dataset_id = 'metadata-sites'
# table_id = 'metadata-sites'

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,state,url
0,Lummi Island Beach Access,"Lummi Island Beach Access, 2198 N Nugent Rd, L...",0x548599933f20ddf9:0x27ab1d69c266db7b,None,48.721761,-122.683155,Tourist attraction,3.7,6,None,None,Open now,https://www.google.com/maps/place//data=!4m2!3...


In [24]:
df.memory_usage(deep = True, index = False)

name              0
address           0
gmap_id           0
description       0
latitude          0
longitude         0
category          0
avg_rating        0
num_of_reviews    0
price             0
hours             0
state             0
url               0
dtype: int64